# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>
- TAs: Tong Zeng <tozeng@syr.edu>, Priya Matnani <psmatnan@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [1]:
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
from pyspark.sql import Row
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Warning: Use exclusively Spark 1.6 when asked to do so and Spark 2.0 (dataframes) only in the last question. Do not use Pandas at all in this assignment

# Part 2: Data cleaning and basic analyses

In this part, you will learn to read data from non-standard formats, clean data, and produce some basic analysis of it.

We will use Spark 1.6 (`sparkContext` on variable `sc`) to load text files from which we will extract features that are predictive of a target value. Unfortunately, the data is stored in some non-standard format where each line contains the customer index, the feature index, and the value of the feature for that customer. Similarly, the target files contain in each line the customer index and the target value. We will load these files into two RDDs:

In [2]:
raw_features_rdd = sc.textFile('hw2_dataset_features.txt')
raw_target_rdd = sc.textFile('hw2_dataset_targets.txt')

An issue with the data is that there were problems transmitting the features and targets. If this happened, then a text `ERROR` or `ERROR TRANSFERRING` replaced the data. If you look at the first 10 values of the features and target RDD, you will see these types of lines:

```python
raw_features_rdd.take(10)
```

```
['<customer_feature customer_id=0 feature_id=0>-0.57</customer_feature>',
 '<customer_feature customer_id=0 feature_id=1>-0.38</customer_feature>',
 '<customer_feature customer_id=0 feature_id=2>0.00</customer_feature>',
 '<customer_feature customer_id=0 feature_id=3>-0.07</customer_feature>',
 '<customer_feature customer_id=0 feature_id=4>-0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=5>-0.79</customer_feature>',
 'ERROR',
 '<customer_feature customer_id=0 feature_id=7>0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=8>1.65</customer_feature>',
 '<customer_feature customer_id=0 feature_id=9>0.57</customer_feature>']
```

```python
raw_target_rdd.take(35)
```

```
['<customer_target customer_id=0>-252.49</customer_target>',
 '<customer_target customer_id=1>36.67</customer_target>',
 '<customer_target customer_id=2>138.02</customer_target>',
 '<customer_target customer_id=3>-429.54</customer_target>',
 '<customer_target customer_id=4>-18.23</customer_target>',
 '<customer_target customer_id=5>-5.52</customer_target>',
 '<customer_target customer_id=6>-31.96</customer_target>',
 'ERROR TRANSFERRING',
 'ERROR TRANSFERRING',
 '<customer_target customer_id=9>-111.88</customer_target>']
```

In [3]:
# try it yourself: raw_features_rdd.take(10) and raw_target_rdd.take(10)
raw_features_rdd.take(10)
raw_target_rdd.take(10)

['<customer_target customer_id=0>-252.49</customer_target>',
 '<customer_target customer_id=1>36.67</customer_target>',
 '<customer_target customer_id=2>138.02</customer_target>',
 '<customer_target customer_id=3>-429.54</customer_target>',
 '<customer_target customer_id=4>-18.23</customer_target>',
 '<customer_target customer_id=5>-5.52</customer_target>',
 '<customer_target customer_id=6>-31.96</customer_target>',
 'ERROR TRANSFERRING',
 'ERROR TRANSFERRING',
 '<customer_target customer_id=9>-111.88</customer_target>']

## Question 2.1 (5 pts):

Filter out the lines that contain errors and store them in `raw_features2_rdd` and `raw_targets_rdd` respectively. 

In [4]:
# create raw_features2_rdd and raw_targets2_rdd below

raw_features2_rdd = raw_features_rdd.filter(lambda x: not "ERROR" in x)
raw_target2_rdd = raw_target_rdd.filter(lambda x: not "ERROR TRANSFERRING" in x)

In [5]:
# check that things work
print(raw_features2_rdd.count())
print(raw_target2_rdd.count())

95036
8968


In [6]:
"""5 pts: Check that the lines are properly discarded"""
assert raw_features2_rdd.count() == 95036
assert raw_target2_rdd.count() == 8968

## Question 2.2 (5 pts):
You will further process `raw_features2_rdd` such that you will create a key-value RDD of the following form: the key is the customer index as an integer and the value is a dictionary whose key is a string `f_0`, `f_1`, ..., `f_9` for feature index 0, 1, ... 9, respetively, and the value is a floating point number of the feature value. 

Define a function `map_features2` that performs such key-value pair creation.

In [7]:
def map_features2(line):
    sections = line.split(" ")
    customer_id = int(sections[1].split("=")[1])
    sections2 = sections[2].split(">")
    feature_id = "f_"+sections2[0].split("=")[1]
    fvalue = float(sections2[1].split("<")[0])
    return [customer_id,{feature_id: fvalue}]

For example, for the input element:

`'<customer_feature customer_id=4 feature_id=0>-0.79</customer_feature>'`

it should generate
```python
[4, {'f_0': -0.79}]
```

In [8]:
# test it here
raw_features2_rdd.\
    map(map_features2).\
    take(10)

[[0, {'f_0': -0.57}],
 [0, {'f_1': -0.38}],
 [0, {'f_2': 0.0}],
 [0, {'f_3': -0.07}],
 [0, {'f_4': -0.28}],
 [0, {'f_5': -0.79}],
 [0, {'f_7': 0.28}],
 [0, {'f_8': 1.65}],
 [0, {'f_9': 0.57}],
 [1, {'f_0': 0.5}]]

In [9]:
"""5 pts: Check that the new raw_features2_rdd and raw_target2_rdd RDDs are correct"""
# key is an integer
np.testing.assert_equal(type(raw_features2_rdd.map(map_features2).first()[0]), int)
# value is a dictionary
np.testing.assert_equal(type(raw_features2_rdd.map(map_features2).first()[1]), dict)

## Question 2.3 (5 pts):

You will create a function `map_target2` that will be applied to `raw_target2_rdd`. This function will create key-value pair where the key is the customer index as an integer and the value is the floating point representation of the target. Assign the resulting RDD to `raw_target3_rdd`. 

In [10]:
def map_target2(line):
    sections = line.split(" ")
    sections2 = sections[1].split(">")
    customer_id = int(sections2[0].split("=")[1])
    value = float(sections2[1].split("<")[0])
    return (customer_id, value)

# make the assignment here
raw_target3_rdd = raw_target2_rdd.map(map_target2)

A sample of results:

```python
raw_target2_rdd.map(map_target2).sortByKey().take(5)
```

```
[(0, -252.49), (1, 36.67), (2, 138.02), (3, -429.54), (4, -18.23)]
```

In [11]:
# try it yourself
raw_target2_rdd.map(map_target2).sortByKey().take(5)

[(0, -252.49), (1, 36.67), (2, 138.02), (3, -429.54), (4, -18.23)]

In [12]:
"""5 pts: Check that raw_target3_rdd contains the right values"""
# check types
np.testing.assert_equal(type(raw_target3_rdd.keys().first()), int)
np.testing.assert_equal(type(raw_target3_rdd.values().first()), float)
# the sum of all targets
np.testing.assert_approx_equal(raw_target3_rdd.values().sum(), -179351.71, significant=1)

## Question 2.4 (10 pts):

In this question, you will use map reduce to produce an RDD of key-value pairs where the key is the customer index and the value is a dictionairy with all the features and values associated with that customer. Notice that the map part of the map-reduce is already defined by `map_features2` on `raw_features2_rdd`. Therefore, define the proper `reduce_features2` function to produce the desired results. Create a RDD named `raw_features3_rdd` with the results:

In [13]:
def reduce_features2(v1, v2):
    z = {**v1, **v2}
    return z


# Apply mapreduce to produce the raw_features3_rdd from raw_features2_rdd
raw_features3_rdd = raw_features2_rdd.map(map_features2).reduceByKey(reduce_features2)

Running the map reduce should produce the following example result:
```python
raw_features3_rdd.sortByKey().take(2)
```

```console
[(0,
  {'f_0': -0.57,
   'f_1': -0.38,
   'f_2': 0.0,
   'f_3': -0.07,
   'f_4': -0.28,
   'f_5': -0.79,
   'f_7': 0.28,
   'f_8': 1.65,
   'f_9': 0.57}),
 (1,
  {'f_0': 0.5,
   'f_1': 0.8,
   'f_2': -0.49,
   'f_3': 0.25,
   'f_4': 0.37,
   'f_5': 0.73,
   'f_6': -0.43,
   'f_7': 0.89,
   'f_8': -1.85,
   'f_9': -0.44})]
```

In [26]:
"""10 pts: Check that raw_features3_rdd has the correct format and values. There could be hidden tests!"""
# key is an integer
np.testing.assert_equal(type(raw_features3_rdd.first()[0]), int)
# value is a dictionary
np.testing.assert_equal(type(raw_features3_rdd.first()[1]), dict)

## Question 2.5 (20 pts):

Join the two RDDs `raw_target3_rdd` and `raw_features3_rdd` to create an RDD with elements of the form

`[customer_index, (target, feature_dict)]`

where `target` comes from `raw_target3_rdd`, and `feature_dict` is the dictionary with features from `raw_features3_rdd`.

Assign to `rdd_complete` a filtered version of the join for customers who have all 10 features and a target.

Finally use `rdd_complete_rows` to create an RDD of `Row` objects where you map each entry to the following format:

`Row(customer_index,  f_0,  f_1,  f_2,  f_3,  f_4,  f_5,  f_6,  f_7,  f_8,  f_9, target)`

For example, from `rdd_complete`:

```python
rdd_complete.sortByKey().first()
```
should return
```
(1,
 (36.67,
  {'f_0': 0.5,
   'f_1': 0.8,
   'f_2': -0.49,
   'f_3': 0.25,
   'f_4': 0.37,
   'f_5': 0.73,
   'f_6': -0.43,
   'f_7': 0.89,
   'f_8': -1.85,
   'f_9': -0.44}))
```

In [32]:
inter = raw_features3_rdd.join(raw_target3_rdd)
rdd_complete = inter.filter(lambda x: len(x[1][0]) == 10) #to check for 10 features


In [33]:
"""10 pts: Test if `rdd1` has the right data. Remember that there could be hidden tests!"""
# number of elements expected
np.testing.assert_equal(rdd_complete.count(), 5379)

In [34]:
# here define the function to_row that a Row object for an entry of rdd_complete
def to_row(e):
    features = e[1][0]
    row = Row(customer_index=e[0],
              f_0 = features.get("f_0"),
              f_1 = features.get("f_1"),
              f_2 = features.get("f_2"),
              f_3 = features.get("f_3"),
              f_4 = features.get("f_4"),
              f_5 = features.get("f_5"),
              f_6 = features.get("f_6"),
              f_7 = features.get("f_7"),
              f_8 = features.get("f_8"),
              f_9 = features.get("f_9"),
              target=e[1][1])
    return row

rdd_complete.map(to_row).toDF().collect()

[Row(customer_index=4, f_0=-0.79, f_1=-0.28, f_2=-0.26, f_3=0.28, f_4=-0.17, f_5=-0.51, f_6=0.44, f_7=-0.62, f_8=1.82, f_9=-0.35, target=-18.23),
 Row(customer_index=12, f_0=-0.09, f_1=-0.26, f_2=-0.95, f_3=0.15, f_4=-0.39, f_5=-0.34, f_6=-0.8, f_7=-0.8, f_8=-1.57, f_9=0.74, target=-356.98),
 Row(customer_index=24, f_0=0.35, f_1=0.63, f_2=0.02, f_3=-0.88, f_4=0.16, f_5=0.78, f_6=-0.34, f_7=-0.06, f_8=1.42, f_9=-0.44, target=21.24),
 Row(customer_index=28, f_0=-0.18, f_1=-1.39, f_2=-0.59, f_3=-0.01, f_4=0.49, f_5=-0.08, f_6=-0.63, f_7=-1.19, f_8=-0.48, f_9=-0.04, target=-234.14),
 Row(customer_index=32, f_0=-1.26, f_1=0.05, f_2=0.5, f_3=0.21, f_4=0.33, f_5=-0.93, f_6=0.09, f_7=1.05, f_8=-0.0, f_9=0.11, target=116.33),
 Row(customer_index=36, f_0=0.2, f_1=0.47, f_2=0.73, f_3=0.59, f_4=-0.28, f_5=0.7, f_6=0.19, f_7=-0.48, f_8=-0.21, f_9=-0.23, target=49.22),
 Row(customer_index=44, f_0=-0.32, f_1=-0.22, f_2=-0.26, f_3=-1.29, f_4=0.2, f_5=-1.32, f_6=0.65, f_7=0.7, f_8=0.12, f_9=0.33, targe

In [35]:
# test
rdd_complete.map(to_row).toDF().orderBy('customer_index').show(5)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows



In [36]:
# 10 pts test that the creation of the row is successful
assert rdd_complete.map(to_row).toDF()
np.testing.assert_array_equal(rdd_complete.map(to_row).toDF().columns, 
              ['customer_index',
 'f_0',
 'f_1',
 'f_2',
 'f_3',
 'f_4',
 'f_5',
 'f_6',
 'f_7',
 'f_8',
 'f_9',
 'target'])

## Question 2.6 (20 pts):

We will now use the `to_row` function created above to create a dataframe of the data `df`

In [20]:
df = rdd_complete.map(to_row).toDF()

Explore the dataframe a bit:

```python
df.orderBy('customer_index').show(5)
```

```console
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows
```

In [21]:
# explore it yourself
df.orderBy('customer_index').show(5)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows



The subpackage `pyspark.ml.functions` (aliased as `fn` below) contains many common functions for data analysis. Find the function for computing the __correlation__ (the strength of the linear relationship between two variables) between two columns, the function for computing __absolute__ values, and create a data frame `correlations_df` that contains the following columns in the following order:

1. `c0_target`: correlation between feature 0 and target
1. `c1_target`: correlation between feature 1 and target
1. `c2_target`: correlation between feature 2 and target
1. `c3_target`: correlation between feature 3 and target
1. `c4_target`: correlation between feature 4 and target
1. `c5_target`: correlation between feature 5 and target
1. `c6_target`: correlation between feature 6 and target
1. `c7_target`: correlation between feature 7 and target
1. `c8_target`: correlation between feature 8 and target
1. `c9_target`: correlation between feature 9 and target
1. `sig0`: boolean `true` if the absolute value of the correlation between feature 0 and target is greater than 0.5, `false` o.w.
1. `sig1`: boolean `true` if the absolute value of the correlation between feature 1 and target is greater than 0.5, `false` o.w.
1. `sig2`: boolean `true` if the absolute value of the correlation between feature 2 and target is greater than 0.5, `false` o.w.
1. `sig3`: boolean `true` if the absolute value of the correlation between feature 3 and target is greater than 0.5, `false` o.w.
1. `sig4`: boolean `true` if the absolute value of the correlation between feature 4 and target is greater than 0.5, `false` o.w.
1. `sig5`: boolean `true` if the absolute value of the correlation between feature 5 and target is greater than 0.5, `false` o.w.
1. `sig6`: boolean `true` if the absolute value of the correlation between feature 6 and target is greater than 0.5, `false` o.w.
1. `sig7`: boolean `true` if the absolute value of the correlation between feature 7 and target is greater than 0.5, `false` o.w.
1. `sig8`: boolean `true` if the absolute value of the correlation between feature 8 and target is greater than 0.5, `false` o.w.
1. `sig9`: boolean `true` if the absolute value of the correlation between feature 9 and target is greater than 0.5, `false` o.w.

**Hint: Remember that you can pass a list of columns to `df.select`. You can create such list with list comprehension, saving a lot of code**

In [22]:
# import the package functions as fn
from pyspark.sql import functions as fn

In [23]:
# apply some function to the columns: df.select(...)

In [37]:
# Create the dataframe `correlations_df` here

c0_target = df.corr(col1 ='f_0', col2 = 'target')
c1_target = df.corr(col1 ='f_1', col2 = 'target')
c2_target = df.corr(col1 ='f_2', col2 = 'target')
c3_target = df.corr(col1 ='f_3', col2 = 'target')
c4_target = df.corr(col1 ='f_4', col2 = 'target')
c5_target = df.corr(col1 ='f_5', col2 = 'target')
c6_target = df.corr(col1 ='f_6', col2 = 'target')
c7_target = df.corr(col1 ='f_7', col2 = 'target')
c8_target = df.corr(col1 ='f_8', col2 = 'target')
c9_target = df.corr(col1 ='f_9', col2 = 'target')

raw_data = [Row(c0_target = c0_target,c1_target = c1_target,
                c2_target = c2_target,c3_target = c3_target,
               c4_target = c4_target,c5_target = c5_target,
                c6_target = c6_target,c7_target = c7_target,
                c8_target = c8_target,c9_target = c9_target          
                )]
intermediate_df = spark.createDataFrame(raw_data)
sig0 = abs(intermediate_df.collect()[0][0])>0.5
sig1 = abs(intermediate_df.collect()[0][1])>0.5
sig2 = abs(intermediate_df.collect()[0][2])>0.5
sig3 = abs(intermediate_df.collect()[0][3])>0.5
sig4 = abs(intermediate_df.collect()[0][4])>0.5
sig5 = abs(intermediate_df.collect()[0][5])>0.5
sig6 = abs(intermediate_df.collect()[0][6])>0.5
sig7 = abs(intermediate_df.collect()[0][7])>0.5
sig8 = abs(intermediate_df.collect()[0][8])>0.5
sig9 = abs(intermediate_df.collect()[0][9])>0.5

raw_data2 = [Row(c0_target = c0_target,c1_target = c1_target,
                c2_target = c2_target,c3_target = c3_target,
               c4_target = c4_target,c5_target = c5_target,
                c6_target = c6_target,c7_target = c7_target,
                c8_target = c8_target,c9_target = c9_target,
                 sig0 = sig0, sig1 = sig1, sig2 = sig2, sig3 = sig3, 
                 sig4 = sig4, sig5 = sig5, sig6 = sig6, sig7 = sig7, 
                 sig8 = sig8, sig9 = sig9
                )]

correlations_df = spark.createDataFrame(raw_data2)

In [38]:
"""20 pts: Check that the dataframe has the correct columns and values. There could be hidden tests!"""
# check column names
column_names = ['c' + str(fi) + '_target' for fi in range(10)] + \
               ['sig' + str(fi) for fi in range(10)]

# column's names and positions in the right order
np.testing.assert_equal(correlations_df.columns, column_names)

from pyspark.sql.types import DoubleType, BooleanType


# the types are correct
np.testing.assert_equal([type(f.dataType) for f in correlations_df.schema.fields], 
                        10*[DoubleType] + 10*[BooleanType])

# the values are correct
np.testing.assert_approx_equal(correlations_df.toPandas().sum().sum(), 
                               4.963039476979365,
                               significant=1)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 45252)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
----------------------------------------
